In [ ]:
import pandas as pd
df = pd.read_excel('/content/drive/Shareddrives/실전기계_project/data_labeling/data_labeling.xlsx', sheet_name=None)
df = pd.concat(df, ignore_index=True)
df = df[['document', 'spo_label']]

In [ ]:
df.head()

,document,spo_label
0,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,0.0
1,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",0.0
2,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,0.0
3,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,0.0
4,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,0.0


In [ ]:
df['spo_label'].value_counts()

1.0    376
0.0    155
Name: spo_label, dtype: int64

In [ ]:
df.isnull().sum()

document        0
spo_label    4469
dtype: int64

In [ ]:
df = df.fillna(0)

In [ ]:
df['spo_label'].value_counts()

0.0    4624
1.0     376
Name: spo_label, dtype: int64

In [ ]:
df_spo = df[df['spo_label']==1]

In [ ]:
import random
import pickle
import re

wordnet = {}
with open("/content/drive/Shareddrives/실전기계_project/wordnet.pickle", "rb") as f:
	wordnet = pickle.load(f)
 
 

In [ ]:
# 한글만 남기고 나머지는 삭제
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)

	return parseText



########################################################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################
def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break

	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")

	else:
		new_words = ""

	return new_words


def get_synonyms(word):
	synomyms = []

	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass

	return synomyms

########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################
def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)

	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]

	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)

	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0

	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words

	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)
	
	return new_words


def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""

		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)


In [ ]:
def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)

	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1

	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))

	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))

	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))

	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))

	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))

	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)

	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

	augmented_sentences.append(sentence)

	return augmented_sentences

In [ ]:
df_spo

,document,spo_label
14,3도 반전 좋았제 ^^,1.0
21,그런 때가 있었다. ('사랑해'도 아니고) 그저 좋아한다는 한 마디 말을 꺼내기도 ...,1.0
25,"7시간짜리 영상이 존재한다면, 죽기전에 꼭 한번 보고싶다. 아름답고 슬픈 OST, ...",1.0
55,정말 최고였는데.. 마지막이.. 이련결말. 싫다..ㅠ,1.0
65,"신부로서의 사명감을 처절히 실천하는 제리, 자신마저 부정하는 로키의 우정",1.0
...,...,...
4930,아버지는 게임 중독에 빠져 코스프레를 하고 형들은 5인조 락밴드,1.0
4934,마지막 장면이 제일 슬펐다.,1.0
4964,"지금보면 좀 오그라드는 장면이 많긴한데, 옛날 당시에 봤을때는 꽤 잼나게 봤던 기억...",1.0
4981,중학교 고등학교시절 세나와 프로스트 만쎌이 F1레이스를 다투던 기사를 보던게 생생하...,1.0


In [ ]:
df_spo_aug = df_spo['document'].apply(EDA)

In [ ]:
aug = []
for i in range(len(df_spo_aug)):
  for j in range(len(df_spo_aug.iloc[i])):
    aug.append(df_spo_aug.iloc[i][j])

In [ ]:
df_augmentation = pd.DataFrame({'document': aug,
             'label': 1})

In [ ]:
df_augmentation

,document,label
0,3도 반전 좋았제 ^^,1
1,3도 반전 좋았제 ^^,1
2,반전 3도 좋았제 ^^,1
3,3도 반전 좋았제 ^^,1
4,^^ 반전 좋았제 3도,1
...,...,...
3755,마지막에 에미 로섬이 마놀로 구두신고 노래부를땐 정말 엔딩곡 감동 ㅠ.ㅠ 폭풍 너무...,1
3756,마지막에 에미 로섬이 마놀로 구두신고 노래부를땐 정말 폭풍 감동 ㅠ.ㅠ 엔딩곡 너무...,1
3757,마지막에 에미 로섬이 마놀로 구두신고 노래부를땐 정말 폭풍 좋네여~!! ㅠ.ㅠ 엔딩...,1
3758,마지막에 에미 로섬이 마놀로 구두신고 노래부를땐 정말 폭풍 감동 ㅠ.ㅠ 엔딩곡 너무...,1


In [ ]:
df_spo_0 = df[df['spo_label'] ==0]

In [ ]:
df_spo_0

,document,spo_label
0,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,0.0
1,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",0.0
2,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,0.0
3,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,0.0
4,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,0.0
...,...,...
4995,이스트우드 옹의 연출능력은 이 영화로 주가가 오르기 시작했다.,0.0
4996,울었습니다 ㅠ..약간 막장드라마끼가 있지만 그래도 만족!,0.0
4997,왜 가슴이 먹먹한가요... ???,0.0
4998,추억이돋는다 돋아..!! 옛날에 본건데 아직도 장수하시고있다니 대단한 인기 !!,0.0


In [ ]:
df_0 = df_spo_0.rename(columns = {'spo_label':'label'})

In [ ]:
data = pd.concat([df_0, df_augmentation])

In [ ]:
data

,document,label
0,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,0.0
1,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",0.0
2,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,0.0
3,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,0.0
4,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,0.0
...,...,...
3755,마지막에 에미 로섬이 마놀로 구두신고 노래부를땐 정말 엔딩곡 감동 ㅠ.ㅠ 폭풍 너무...,1.0
3756,마지막에 에미 로섬이 마놀로 구두신고 노래부를땐 정말 폭풍 감동 ㅠ.ㅠ 엔딩곡 너무...,1.0
3757,마지막에 에미 로섬이 마놀로 구두신고 노래부를땐 정말 폭풍 좋네여~!! ㅠ.ㅠ 엔딩...,1.0
3758,마지막에 에미 로섬이 마놀로 구두신고 노래부를땐 정말 폭풍 감동 ㅠ.ㅠ 엔딩곡 너무...,1.0


In [ ]:
data = data.sample(frac=1)

In [ ]:
data.to_csv("/content/drive/Shareddrives/실전기계_project/data_spo.csv")